Deep Learning in het Nederlands
=============

Tensorflow in Dutch
------------
In de reeks `Deep Learning in het Nederlands - Tensorflow in Dutch` maak je kennis met de methodes en technieken voor Deep Learning met Tensorflow. Experimenteer, duik in het diepe of leer bij met de 6 opdrachten. De volledige repository staat op mijn [Github](https://github.com/victorr0/Deep-Learning-Nederlands). 

TensorFlow is een open source software bibliotheek voor Machine Learning waarmee je neurale netwerken kunt bouwen en trainen om patronen en correlaties te detecteren en te ontcijferen: [Tensorflow](https://github.com/tensorflow/tensorflow.git). 

Deze opdrachten volgen het leertraject van [Udacity](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/udacity) en zijn in het Engels beschikbaar in de standaard TensorFlow repository. Ik heb ze naar het Nederlands vertaald en in een repository geplaatst. 


Opdracht 2
------------

In de vorige opdracht 1_notmnist.ipynb hebben we een pickle-bestand gemaakt met de geformatteerde datasets voor de training, ontwikkeling en het testen met de [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset.

Het doel van deze opdracht is om stapsgewijs modellen te trainen die dieper en nauwkeuriger worden met behulp van TensorFlow.

In [0]:
# Dit zijn alle modules die nodig zijn voor dit notebook. 
# Zorg dat ze allemaal geimporteerd kunnen worden
# voordat je verder gaat.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

Eerst laden we opnieuw de data uit `1_notmnist.ipynb`.

In [0]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # Hint om geheugen vrij te maken.
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Nu formatteren we de data zodat die beter past bij de modellen die we gaan trainen:
- data als een flat matrix,
- labels als float 1-hot encodings.

In [0]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map van 0 naar [1.0, 0.0, 0.0 ...], 1 naar [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


We gaan eerst een multinomiale logistieke regressie trainen met behulp van simple gradient descent.

TensorFlow werkt als volgt:
* Eerst beschrijf je de berekening die uitgevoerd moet worden: hoe de inputs, de variabelen en de operations eruit moeten zien. Die worden gecreëerd als nodes (knooppunten) over een berekeningsgrafiek. Deze beschrijving wordt omvat in het onderstaande blok:

      with graph.as_default():
          ...

* Vervolgens kun je de bewerkingen op deze grafiek zo vaak uitvoeren als je wilt door `session.run()` aan te roepen. Je krijgt dan de output uit de grafiek terug. Deze runtime-operatie wordt omvat in het onderstaande blok:

      with tf.Session(graph=graph) as session:
          ...

Nu laden we alle data in Tensorflow en bouwen de berekeningsgrafiek op die overeenkomt met onze training:

In [0]:
# Met gradiënte afdaling (gradient descent) training is deze berekening computationeel erg zwaar.
# Daarom verdelen we de training data, met een snellere turnaround (omkeertijd) als resultaat.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Laad de training, validation and test data naar constanten
  # gekoppeld aan de grafiek.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variabelen.
  # Dit zijn de parameters die we gaan trainen. De gewicht
  # matrix (weight) wordt geïnitialiseerd met behulp van willekeurige waarden 
  # volgens een (verkorte) normale verdeling. De biases
  # worden geïnitialiseerd naar nul.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training berekeningen.
  # We vermenigvuldigen de invoer met de gewichtsmatrix en voegen de bias toe.
  # We berekenen de softmax en cross-entropy (dit is een operation in TensorFlow, 
  # omdat het heel vaak voorkomt, en het kan worden geoptimaliseerd).
  # We nemen het gemiddelde van deze cross-entropie over alle trainingsvoorbeelden: 
  # dat is ons verlies (loss).
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimalisator (Optimizer).
  # We gaan het minimum van dit verlies vinden door gebruik te maken van gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Voorspellingen  (predictions) voor de training, validation en test data.
  # Deze zijn niet onderdeel van training, maar staan hier alleen zodat we 
  # nauwkeurigheidscijfers kunnen rapporteren terwijl we trainen.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Laten we deze berekening uitvoeren en itereren (herhalen):

In [0]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # Dit is een eenmalige operatie die ervoor zorgt dat de parameters worden geïnitialiseerd 
  # zoals we in de grafiek beschrijven: willekeurige gewichten (weights) voor de matrix, nullen (zeros) 
  # voor de voorspellingen.
  tf.global_variables_initializer().run()
  print('Geïnitialiseerd')
  for step in range(num_steps):
  # Voer de berekeningen uit. We vertellen .run () dat we de optimizer willen uitvoeren 
  # en de verlieswaarde en de trainingsvoorspellingen terug willen als numpy arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Verlies bij stap %d: %f' % (step, l))
      print('Training nauwkeurigheid: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Het aanroepen van .eval() op valid_prediction is in principe hetzelfde als .run() aanroepen, 
      # maar alleen om die ene numpy array te krijgen. Merk op dat het al de grafische 
      # afhankelijkheden opnieuw berekent.
      print('Validation nauwkeurigheid: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test nauwkeurigheid: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0 : 17.2939
Training accuracy: 10.8%
Validation accuracy: 13.8%
Loss at step 100 : 2.26903
Training accuracy: 72.3%
Validation accuracy: 71.6%
Loss at step 200 : 1.84895
Training accuracy: 74.9%
Validation accuracy: 73.9%
Loss at step 300 : 1.60701
Training accuracy: 76.0%
Validation accuracy: 74.5%
Loss at step 400 : 1.43912
Training accuracy: 76.8%
Validation accuracy: 74.8%
Loss at step 500 : 1.31349
Training accuracy: 77.5%
Validation accuracy: 75.0%
Loss at step 600 : 1.21501
Training accuracy: 78.1%
Validation accuracy: 75.4%
Loss at step 700 : 1.13515
Training accuracy: 78.6%
Validation accuracy: 75.4%
Loss at step 800 : 1.0687
Training accuracy: 79.2%
Validation accuracy: 75.6%
Test accuracy: 82.9%


Laten we nu switchen naar stochastic gradient descent training, iets dat veel sneller werkt!

De grafiek zal vergelijkbaar zijn, behalve dat in plaats van alle trainingsgegevens in een constante node te houden, creëren we een `Placeholder`-node, dat de daadwerkelijke gegevens zal invoeren bij elke aanroep van ` session.run() `.

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. Voor de trainings data gebruiken we een plaatshouder 
  # die tijdens de het uitvoeren wordt gevoed met een training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variabelen
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training berekeningen.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimalisator (Optimizer).
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Voorspellingen voor de training, validation en test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Dan is het nu tijd om uit te voeren:

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Kies een compensatie (offset) voor de trainings data, die gerandomiseerd zijn.
    # Opmerking: we kunnen betere randomisatie gebruiken over epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Genereer een minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Bereid een dictionary voor die de sessie aangeeft waar de minibatch heen moet.
    # De key in de dictionary is de plaatshouder-node van de grafiek die moet worden ingevoerd, 
    # en de value is de numpy array die vervolgens in de grafiek wordt gevoerd.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch verlies bij stap %d: %f" % (step, l))
      print("Minibatch nauwkeurigheid: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation nauwkeurigheid: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test nauwkeurigheid: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 16.8091
Minibatch accuracy: 12.5%
Validation accuracy: 14.0%
Minibatch loss at step 500 : 1.75256
Minibatch accuracy: 77.3%
Validation accuracy: 75.0%
Minibatch loss at step 1000 : 1.32283
Minibatch accuracy: 77.3%
Validation accuracy: 76.6%
Minibatch loss at step 1500 : 0.944533
Minibatch accuracy: 83.6%
Validation accuracy: 76.5%
Minibatch loss at step 2000 : 1.03795
Minibatch accuracy: 78.9%
Validation accuracy: 77.8%
Minibatch loss at step 2500 : 1.10219
Minibatch accuracy: 80.5%
Validation accuracy: 78.0%
Minibatch loss at step 3000 : 0.758874
Minibatch accuracy: 82.8%
Validation accuracy: 78.8%
Test accuracy: 86.1%


---
Probleem/vraag
-------

Verander het logistic regression voorbeeld met SGD naar een 1-hidden layer (verborgen laag) neural network met  rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) en 1024 hidden nodes (knooppunten). Dit model moet je validatie- en test-nauwkeurigheid verbeteren.

---